## Home assignment 05: Bagging and OOB score

Please, fill the lines in the code below.
This is a simplified version of `BaggingRegressor` from `sklearn`. Please, notice, that `sklearn` API is **not preserved**.

Your algorithm should be able to train different instances of the same model class on bootstrapped datasets and to provide [OOB score](https://en.wikipedia.org/wiki/Out-of-bag_error) for the training set.

The model should be passed as model class with no explicit parameters and no parentheses.

Example:
```
import numpy as np
from sklearn.linear_model import LinearRegression

bagging_regressor = SimplifiedBaggingRegressor(num_bags=10, oob=True)
bagging_regressor.fit(LinearRegression, X, y)

```

In [ ]:
import numpy as np

class SimplifiedBaggingRegressor:
    def __init__(self, num_bags, oob=False):
        self.num_bags = num_bags
        self.oob = oob
        
    def _generate_splits(self, data: np.ndarray):
        self.indices_list = []
        data_length = len(data)
        for _ in range(self.num_bags):
            bootstrap_indices = np.random.choice(data_length, data_length, replace=True)
            self.indices_list.append(bootstrap_indices)
        
    def fit(self, model_constructor, data, target):
        self.data = None
        self.target = None
        self._generate_splits(data)

        
        self.models_list = []
        for bootstrap_indices in self.indices_list:
            model = model_constructor()
            data_bag = data[bootstrap_indices]
            target_bag = target[bootstrap_indices]
            self.models_list.append(model.fit(data_bag, target_bag))
        
        if self.oob:
            self.data = data
            self.target = target
        
    def predict(self, data):

        predictions = np.array([model.predict(data) for model in self.models_list])
        return predictions.mean(axis=0)
    
    def _get_oob_predictions_from_every_model(self):

        list_of_predictions_lists = [[] for _ in range(len(self.data))]
        
        for model, bootstrap_indices in zip(self.models_list, self.indices_list):
            oob_mask = np.ones(len(self.data), dtype=bool)
            oob_mask[bootstrap_indices] = False
            oob_data = self.data[oob_mask]
            oob_indices = np.where(oob_mask)[0]
            
            if len(oob_data) > 0:
                oob_predictions = model.predict(oob_data)
                for index, pred in zip(oob_indices, oob_predictions):
                    list_of_predictions_lists[index].append(pred)
        
        self.list_of_predictions_lists = np.array(list_of_predictions_lists, dtype=object)
    
    def _get_averaged_oob_predictions(self):

        self._get_oob_predictions_from_every_model()
        self.oob_predictions = np.array([
            np.mean(predictions) if len(predictions) > 0 else np.nan
            for predictions in self.list_of_predictions_lists
        ])

        
    def OOB_score(self):

        self._get_averaged_oob_predictions()
        valid_predictions_mask = ~np.isnan(self.oob_predictions)
        mse = np.mean((self.target[valid_predictions_mask] - self.oob_predictions[valid_predictions_mask])**2)
        return mse


### Local tests:

In [21]:
from sklearn.linear_model import LinearRegression
from tqdm.auto import tqdm

#### Simple tests:

In [24]:
for _ in tqdm(range(100)):
    X = np.random.randn(2000, 10)
    y = np.mean(X, axis=1)
    bagging_regressor = SimplifiedBaggingRegressor(num_bags=10, oob=True)
    bagging_regressor.fit(LinearRegression, X, y)
    predictions = bagging_regressor.predict(X)
    assert np.mean((predictions - y)**2) < 1e-6, 'Linear dependency should be fitted with almost zero error!'
    assert bagging_regressor.oob, 'OOB feature must be turned on'
    oob_score = bagging_regressor.OOB_score()
    assert oob_score < 1e-6, 'OOB error for linear dependency should be also close to zero!'
    assert abs(
        np.mean(
            list(map(len, bagging_regressor.list_of_predictions_lists))
        ) / bagging_regressor.num_bags - 1/np.exp(1)) < 0.1, 'Probability of missing a bag should be close to theoretical value!'
    
print('Simple tests done!')

100%|██████████| 100/100 [00:02<00:00, 49.21it/s]

Simple tests done!


#### Medium tests

In [25]:
for _ in tqdm(range(10)):
    X = np.random.randn(200, 150)
    y = np.random.randn(len(X))
    bagging_regressor = SimplifiedBaggingRegressor(num_bags=20, oob=True)
    bagging_regressor.fit(LinearRegression, X, y)
    predictions = bagging_regressor.predict(X)
    average_train_error = np.mean((predictions - y)**2)
    assert bagging_regressor.oob, 'OOB feature must be turned on'
    oob_score = bagging_regressor.OOB_score()
    assert oob_score > average_train_error, 'OOB error must be higher than train error due to overfitting!'
    assert abs(
        np.mean(
            list(map(len, bagging_regressor.list_of_predictions_lists))
        ) / bagging_regressor.num_bags - 1/np.exp(1)) < 0.1, 'Probability of missing a bag should be close to theoretical value!'
    
print('Medium tests done!')

100%|██████████| 10/10 [00:08<00:00,  1.14it/s]

Medium tests done!


#### Complex tests:

In [26]:
for _ in tqdm(range(10)):
    X = np.random.randn(2000, 15)
    y = np.random.randn(len(X))
    bagging_regressor = SimplifiedBaggingRegressor(num_bags=100, oob=True)
    bagging_regressor.fit(LinearRegression, X, y)
    predictions = bagging_regressor.predict(X)
    oob_score = bagging_regressor.OOB_score()
    assert abs(
        np.mean(
            list(map(len, bagging_regressor.list_of_predictions_lists))
        ) / bagging_regressor.num_bags - 1/np.exp(1)) < 1e-2, 'Probability of missing a bag should be close to theoretical value!'
    
print('Complex tests done!')

100%|██████████| 10/10 [00:01<00:00,  5.19it/s]

Complex tests done!


In [27]:
np.mean(
            list(map(len, bagging_regressor.list_of_predictions_lists))
        ) / bagging_regressor.num_bags - 1/np.exp(1)

0.00027055882855764324

Great job! Please, save `SimplifiedBaggingRegressor` to  `bagging.py` and submit your solution to the grading system!